## 抓取MPC上小行星的数据
作者：AstroHHQ 初步判断是否有wise数据

### request 实现

load import & minorplanet list

In [24]:
#load import
import numpy as np
import pandas as pd
from urllib.request import urlopen
from requests.api import request

mpcdata = pd.read_table('mpc.txt')
#mpcdata.sort_values('Diam',ascending=False)
sort_data = mpcdata.sort_values('Diam',ascending=False)
#mpcdata['Name'][0]
mp_name = mpcdata.loc[:,['Name','Diam']]
mp_name.insert(2,'WiseData_time',0)
mp_name


,Name,Diam,WiseData_time
0,1991 VA,0.017,0
1,1994 CJ1,0.156,0
2,1995 CR,0.130,0
3,1995 HM,0.094,0
4,1996 GQ,0.071,0
...,...,...,...
1310,2011 LT17,0.134,0
1311,2014 AY28,0.122,0
1312,2014 QX266,0.091,0
1313,2015 BY310,0.130,0


creat a new list of minorplanet(record is there wise data and date)

In [34]:
import requests as rq
from tqdm import tqdm
#space to "+"
#name = mp_name[5]
#name = name.replace(' ','+')
lenMp = len(mp_name)
total = 0
for i in tqdm(range(lenMp)):
   name = mp_name['Name'][i]
   name = name.replace(' ','+')
   #get url from mpc
   url = f'https://minorplanetcenter.net/db_search/show_object?utf8=%E2%9C%93&object_id={name}'
   head ={
      'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:94.0) Gecko/20100101 Firefox/94.0'
   }
   resp = rq.get(url,headers=head)
   #get html table
   mptable = pd.read_html(resp.text,match='Location')
   df = mptable[0]
   #is there wise data?
   dfyes = df[df['Location']=='C51 – WISE']
   if len(dfyes)>=1:
      dfout = dfyes["Date (UT)"]
      total = total + 1
      #get 1st date
      mp_name['WiseData_time'][i] = dfout[dfout.index[0]]
   else:
      mp_name['WiseData_time'][i] = -1
print(f'{total} wise data in {lenMp}') 
mp_name  


  1%|          | 16/1315 [00:29<40:24,  1.87s/it]


ValueError: No tables found matching pattern 'Location'

In [40]:
mp_name.loc[0:17]

,Name,Diam,WiseData_time
0,1991 VA,0.017,-1
1,1994 CJ1,0.156,-1
2,1995 CR,0.130,-1
3,1995 HM,0.094,-1
4,1996 GQ,0.071,2010 02 05.963786
5,1996 VB3,0.102,-1
6,1997 UR,0.068,-1
7,1998 BY7,0.151,-1
8,1998 KY26,0.024,-1
9,1998 WB2,0.113,-1


上面发现只有16号名字不符合 ，所以后面从第17个开始循环,total = 2

In [41]:
import requests as rq
from tqdm import tqdm
import time
#space to "+"
#name = mp_name[5]
#name = name.replace(' ','+')
lenMp = len(mp_name)
total = 2
for i in tqdm(range(17,lenMp)):
   if i%20==0:time.sleep(1)
   name = mp_name['Name'][i]
   name = name.replace(' ','+')
   #get url from mpc
   url = f'https://minorplanetcenter.net/db_search/show_object?utf8=%E2%9C%93&object_id={name}'
   head ={
      'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:94.0) Gecko/20100101 Firefox/94.0'
   }
   resp = rq.get(url,headers=head)
   #get html table
   mptable = pd.read_html(resp.text,match='Location')
   df = mptable[0]
   #is there wise data?
   dfyes = df[df['Location']=='C51 – WISE']
   if len(dfyes)>=1:
      dfout = dfyes["Date (UT)"]
      total = total + 1
      #get 1st date
      mp_name['WiseData_time'][i] = dfout[dfout.index[0]]
   else:
      mp_name['WiseData_time'][i] = -1
print(f'{total} wise data in {lenMp}') 
mp_name  


 92%|█████████▏| 1192/1298 [32:41<02:54,  1.65s/it]


ValueError: No tables found matching pattern 'Location'

In [43]:
mp_name.loc[1200:1220]

,Name,Diam,WiseData_time
1200,2021 GQ10,0.014,-1
1201,2021 HC3,0.143,-1
1202,2021 HN,0.012,-1
1203,2021 JB6,0.005,-1
1204,2021 JR3,0.030,-1
1205,2021 KN2,0.006,-1
1206,2021 NY1,0.119,-1
1207,2021 RS,0.027,-1
1208,2021 UH2,0.026,-1
1209,333P/LINEAR,NaN,0


1209-1215号名称不是标准格式，以下从1216开始搜索

In [49]:
mp_name.loc[1240:1246]

,Name,Diam,WiseData_time
1240,P001GV,NaN,0
1241,P002I1,NaN,0
1242,P002I6,NaN,0
1243,P002RZ,NaN,0
1244,YORP,0.086,0
1245,Mjolnir,0.142,0
1246,2003 MA3,0.136,0


从1246开始搜索

In [50]:
lenMp = len(mp_name)
total = 0
for i in tqdm(range(1246,lenMp)):
   if i%20==0:time.sleep(1)
   name = mp_name['Name'][i]
   name = name.replace(' ','+')
   #get url from mpc
   url = f'https://minorplanetcenter.net/db_search/show_object?utf8=%E2%9C%93&object_id={name}'
   head ={
      'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:94.0) Gecko/20100101 Firefox/94.0'
   }
   resp = rq.get(url,headers=head)
   #get html table
   mptable = pd.read_html(resp.text,match='Location')
   df = mptable[0]
   #is there wise data?
   dfyes = df[df['Location']=='C51 – WISE']
   if len(dfyes)>=1:
      dfout = dfyes["Date (UT)"]
      total = total + 1
      #get 1st date
      mp_name['WiseData_time'][i] = dfout[dfout.index[0]]
   else:
      mp_name['WiseData_time'][i] = -1
print(f'{total} wise data in {lenMp}') 
mp_name  


 52%|█████▏    | 36/69 [01:22<01:15,  2.30s/it]


ValueError: No tables found matching pattern 'Location'

从1283开始

In [55]:
mp_name.loc[1280:1283]

,Name,Diam,WiseData_time
1280,2004 KH17,0.124,-1
1281,2014 KQ76,0.136,-1
1282,Kamo`oalewa,0.041,0
1283,2002 LT24,0.116,0


In [56]:
lenMp = len(mp_name)
total = 0
for i in tqdm(range(1283,lenMp)):
   if i%20==0:time.sleep(1)
   name = mp_name['Name'][i]
   name = name.replace(' ','+')
   #get url from mpc
   url = f'https://minorplanetcenter.net/db_search/show_object?utf8=%E2%9C%93&object_id={name}'
   head ={
      'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:94.0) Gecko/20100101 Firefox/94.0'
   }
   resp = rq.get(url,headers=head)
   #get html table
   mptable = pd.read_html(resp.text,match='Location')
   df = mptable[0]
   #is there wise data?
   dfyes = df[df['Location']=='C51 – WISE']
   if len(dfyes)>=1:
      dfout = dfyes["Date (UT)"]
      total = total + 1
      #get 1st date
      mp_name['WiseData_time'][i] = dfout[dfout.index[0]]
   else:
      mp_name['WiseData_time'][i] = -1
print(f'{total} wise data in {lenMp}') 
mp_name  


100%|██████████| 32/32 [01:11<00:00,  2.24s/it]

8 wise data in 1315


,Name,Diam,WiseData_time
0,1991 VA,0.017,-1
1,1994 CJ1,0.156,-1
2,1995 CR,0.130,-1
3,1995 HM,0.094,-1
4,1996 GQ,0.071,2010 02 05.963786
...,...,...,...
1310,2011 LT17,0.134,-1
1311,2014 AY28,0.122,-1
1312,2014 QX266,0.091,-1
1313,2015 BY310,0.130,-1


统计完毕，输出excel

In [59]:
mp_name.to_csv('wisetime.txt')

统计出现次数

In [77]:
df = mp_name
df['WiseData_time'].value_counts()
df.to_csv('wise.txt',sep='\t',index=False)
df.to_excel('wixe.xlsx')